
# 📡 RF Kalibratie + Positievisualisatie (3× Raspberry Pi)

Deze notebook combineert:
1. **Kalibratie en ruisbepaling** van drie Raspberry Pi's  
2. **Interactieve visualisatie** van de Pi-posities en de geschatte GSM-positie met logaritmisch afstandsmodel  

De parameters (C, n, σ, ...) worden automatisch ingeladen uit `rf_calibration_params.json` en kunnen **niet aangepast** worden.  
Je kunt wél de posities van de Pi's en GSM met sliders wijzigen om de geometrie en cirkels te verkennen.


In [1]:

import json
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, fixed

# Inladen van eerder berekende parameters
with open("rf_calibration_params.json", "r") as f:
    params = json.load(f)

combined = params["combined"]
per_pi = params["per_pi"]

print("✅ Parameters geladen uit rf_calibration_params.json")

# We tonen ze mooi in een tabelvorm
import pandas as pd
df_params = pd.DataFrame(per_pi).T
display(df_params.round(4))

print("\nGecombineerd model:")
for k, v in combined.items():
    print(f"{k:25s}: {v:.4f}")


✅ Parameters geladen uit rf_calibration_params.json


,C_intercept,slope_c1,path_loss_n,sigma_residual_db,R2,sigma_duplicate_db
Pi1,-51.6618,-13.6821,1.3682,2.4391,0.7958,0.1904
Pi2,-51.3403,-14.0172,1.4017,2.3621,0.8135,0.1785
Pi3,-51.7568,-13.6343,1.3634,2.5185,0.7840,0.1436



Gecombineerd model:
C_intercept              : -51.5863
slope_c1                 : -13.7779
path_loss_n              : 1.3778
sigma_residual_db        : 2.3355
R2                       : 0.7974
sigma_duplicate_mean_db  : 0.1708


In [2]:

# Model: RSSI(d) = C - 10*n*log10(d)
def predicted_rssi(d, C, n):
    return C - 10 * n * np.log10(d)

# Omgekeerd: afstand uit RSSI
def distance_from_rssi(rssi, C, n):
    return 10 ** ((C - rssi) / (10 * n))


In [10]:

# 🧭 Interactieve positievisualisatie
# ------------------------------------------------------------
# - Je kunt de posities van Pi1, Pi2, Pi3 en GSM verschuiven met sliders.
# - De cirkels geven de geschatte afstand weer volgens het gecombineerde model.
# - De parameters (C, n) worden uit rf_calibration_params.json gehaald en niet aangepast.

# Default startposities (in meter)
pi_positions_default = {
    "Pi1": (0.0, 0.0),
    "Pi2": (5.0, 0.0),
    "Pi3": (2.5, 4.0)
}
gsm_default = (2.5, 2.0)

C = combined["C_intercept"]
n = combined["path_loss_n"]

def plot_positions(pi1_x, pi1_y, pi2_x, pi2_y, pi3_x, pi3_y, gsm_x, gsm_y):
    """
    Toon Pi's, GSM, afstandscirkels met foutmarge én rode 'estimated detection area'.
    Fout op de rode cirkelstraal wordt onder de grafiek weergegeven.
    """
    plt.figure(figsize=(6,6))
    pis = {
        "Pi1": (pi1_x, pi1_y),
        "Pi2": (pi2_x, pi2_y),
        "Pi3": (pi3_x, pi3_y),
    }
    gsm = (gsm_x, gsm_y)

    # Parameters uit kalibratie
    C = combined["C_intercept"]
    n = combined["path_loss_n"]
    sigma_db = combined["sigma_residual_db"]

    est_xs, est_ys, d_errs = [], [], []

    for name, (x, y) in pis.items():
        # Afstand en fout
        d = np.sqrt((gsm_x - x)**2 + (gsm_y - y)**2)
        d_err = d * (10 ** (sigma_db / (10 * n)) - 1)
        d_errs.append(d_err)

        # Hoofdafstand + foutbanden
        plt.scatter(x, y, label=name, s=100)
        plt.gca().add_patch(plt.Circle((x, y), d, color="gray", alpha=0.25))
        plt.gca().add_patch(plt.Circle((x, y), d + d_err, color="gray", alpha=0.1, linestyle="--"))
        plt.gca().add_patch(plt.Circle((x, y), max(0, d - d_err), color="gray", alpha=0.1, linestyle="--"))
        plt.text(x + 0.1, y + 0.1, f"{name}", fontsize=8)
        est_xs.append(x)
        est_ys.append(y)

    # GSM (werkelijke positie)
    plt.scatter(gsm_x, gsm_y, color="red", label="GSM", s=100, marker="x")
    plt.text(gsm_x + 0.1, gsm_y + 0.1, "GSM", color="red", fontsize=9)

    # 🔴 Geschatte zone: gemiddelde van Pi-posities
    est_cx = np.mean(est_xs)
    est_cy = np.mean(est_ys)
    est_r = np.mean(d_errs)

    plt.gca().add_patch(plt.Circle((est_cx, est_cy), est_r, color="red", alpha=0.15, lw=2))
    plt.text(est_cx + 0.2, est_cy + 0.2, "Estimated GSM zone", color="red", fontsize=8)

    plt.title("Positie van Pi's, GSM en geschatte detectiezone (met ruis)")
    plt.xlabel("x [m]")
    plt.ylabel("y [m]")
    plt.axis("equal")
    plt.grid(True)
    plt.legend()
    plt.show()

    # 🧾 Toon fout onder de grafiek
    avg_err = np.mean(d_errs)
    print(f"Gemiddelde straalfout op geschatte detectiezone: ±{avg_err:.3f} meter")

# 🔧 Sliders met schaal van -10 tot 10 meter
interact(
    plot_positions,
    pi1_x=FloatSlider(value=0.0, min=-10, max=10, step=0.1, description="Pi1 x"),
    pi1_y=FloatSlider(value=0.0, min=-10, max=10, step=0.1, description="Pi1 y"),
    pi2_x=FloatSlider(value=5.0, min=-10, max=10, step=0.1, description="Pi2 x"),
    pi2_y=FloatSlider(value=0.0, min=-10, max=10, step=0.1, description="Pi2 y"),
    pi3_x=FloatSlider(value=2.5, min=-10, max=10, step=0.1, description="Pi3 x"),
    pi3_y=FloatSlider(value=4.0, min=-10, max=10, step=0.1, description="Pi3 y"),
    gsm_x=FloatSlider(value=2.5, min=-10, max=10, step=0.1, description="GSM x"),
    gsm_y=FloatSlider(value=2.0, min=-10, max=10, step=0.1, description="GSM y"),
);



interactive(children=(FloatSlider(value=0.0, description='Pi1 x', max=10.0, min=-10.0), FloatSlider(value=0.0,…


## 📘 Uitleg bij de visualisatie

- De **drie blauwe punten** zijn de Raspberry Pi’s.  
- De **rode ‘x’** is de GSM.  
- Elke **grijze cirkel** stelt de afstand voor tussen een Pi en de GSM.  
  (In een echte simulatie zouden die afstanden uit het RSSI-model komen.)

Met de **sliders** kun je de posities aanpassen en zien hoe de geometrie verandert.  
De parameters `C`, `n`, `σ` worden weergegeven maar niet gewijzigd — die komen uit de kalibratie.
